In [135]:
import os
import gmpy2
import hashlib
import sympy
from sympy.ntheory.modular import crt

In [80]:
#Assumes we are given Message, p, q and secret keys
Message = "slifhwenjfnewnfefwelafafeflmealfm32mflm23pfm3p2mf3m"
p = 776013677466283374829664077979289607703797802248972134730822459666549512547789492942820732883494078514296773729111709915350000261085021001313159067
#This is an insecure form for q. It should be p = 2q + 1 but this is for another exercise
q = p - 1
#a is the generator
a = 2
bits = 128

In [81]:
#This is the private key
#not sure why exactly, but the run time is nasty in this version 
#seems like it's too difficult to find a 512 bit random number smaller than q
secret = int.from_bytes(os.urandom(int(bits/8)),byteorder='big')
while secret >= (q-1):
    secret = int.from_bytes(os.urandom(int(bits/8)),byteorder='big')

In [82]:
#This is a^-s mod p for the public key
pub = gmpy2.powmod(a, -secret, p)


In [83]:
def signature_generation(M,p,q,a,s,bits):
    #computing e as part of the signature
    #r is a random number less than q
    r = int.from_bytes(os.urandom(int(bits/8)),byteorder='big')
    while r > (q-1):
        r = int.from_bytes(os.urandom(int(bits/8)),byteorder='big')

    x = pow(a,r,p)

    hash_string = M + str(x)
    e = hashlib.sha256(hash_string.encode()).hexdigest()

    #computing y as second part of the signature
    y = (r + s*int(e,16)) % q

    return int(e,16),int(y)

In [84]:
(e,y) = signature_generation(Message, p, q, a, secret, bits)

### Now it's Bob's turn to validate the signature

In [85]:
# First calculate x_prime using the public info.
x_prime = (gmpy2.powmod(a,y,p) * gmpy2.powmod(pub,e,p)) % p

In [86]:
#Convert H(M+x_prime) for verification against H(M+x)
hash_string = Message + str(x_prime)
verify = hashlib.sha256(hash_string.encode()).hexdigest()
verify = int(verify,16)

In [87]:
if verify == e:
    print('Success! You verified the signature!')
else:
    print('You failed!!')

Success! You verified the signature!


### Now we can see about cracking the discrete log problem with Pohlig-Hellman

As a reminder the following is public information:

1. a the generator
2. pub the public key
3. p the prime
4. e the hashed and concatenated message
5. y the signature
6. m the original message

This application is insecure because **q = p - 1** which means that we can factor q

In [88]:
factor_q = sympy.factorint(q)

In [89]:
print(factor_q)

{2: 1, 784242601: 1, 436841551: 1, 977367739: 1, 792164221: 1, 152736893: 1, 44776547: 1, 158061997: 1, 1066198663: 1, 884591369: 1, 629535299: 1, 944703961: 1, 297674633: 1, 147755633: 1, 478406879: 1, 290264287: 1, 487523821: 1, 810215191: 1}


In [90]:
#Calculate the Moduli
factor_q_list = []
for key in factor_q:
    factor_q_list.append(key**factor_q[key])

In [91]:
print(factor_q_list)

[2, 784242601, 436841551, 977367739, 792164221, 152736893, 44776547, 158061997, 1066198663, 884591369, 629535299, 944703961, 297674633, 147755633, 478406879, 290264287, 487523821, 810215191]


In [138]:
#This function needs optimized. It's running too long.
#It'll solve, but needs to run for hours. Baby step, giant step would be a more practical approach
def find_x(y,g,q,q_factors,p):
    residues = []
    length = len(q_factors)
    for i in range(length):
        z = 0
        found = False
        while not found:
            y_1 = gmpy2.powmod(y,q//q_factors[i],p)
            g_1 = gmpy2.powmod(g,q//q_factors[i],p)
            if y_1 == gmpy2.powmod(g_1,z,p):
                found = True
                residues.append(z)
                print(z)
            z += 1
            if z == q:
                print("Failed to find x")
                break
    return residues

In [137]:
residue_list = find_x(pub,a,p-1,factor_q_list,p)

194003419366570843707416019494822401925949450562243033682705614916637378136947373235705183220873519628574193432277927478837500065271255250328289766


KeyboardInterrupt: 

In [ ]:
x_found = sympy.ntheory.modular.crt(factor_q_list,residue_list)

In [ ]:
print(x_found)